

## 1. **Import Required Libraries**
- **`dotenv`**: Used to load environment variables from a `.env` file, which is useful for keeping sensitive data like API keys secure.
- **`os`**: For interacting with the operating system, particularly to access environment variables.
- **`crewai`**: Provides core functionality for creating agents, tasks, and workflows (referred to as "Crew") for the multi-agent system.
- **`requests`**: Used to make HTTP requests, likely for the web crawler to fetch data.
- **Custom agents**: The `WebCrawlerAgent`, `SummarizerAgent`, and `ReportGeneratorAgent` are imported from the `agents` folder. These agents perform tasks like web scraping, summarization, and report generation.
- **`gradio`**: A library to build and deploy machine learning interfaces quickly.





In [ ]:
from dotenv import load_dotenv
import os
from crewai import Agent, Crew, Task, LLM, Process
import requests
from agents.web_crawler_agent import WebCrawlerAgent
from agents.summarizer_agent import SummarizerAgent
from agents.report_generator_agent import ReportGeneratorAgent
import gradio as gr


## 2. **Load Environment Variables**
The `load_dotenv()` function loads environment variables from a `.env` file. The API key for the Serper service is retrieved from the environment using `os.getenv()` to keep sensitive data secure.

---

## 3. **LLM (Large Language Model) Initialization**
An instance of an LLM (Large Language Model) is initialized using the `ollama/llama3.2` model, hosted on a local server. This model will be used by the agents for various tasks.


In [42]:

load_dotenv()
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# Create an LLM instance for Ollama LLaMA 3.2
llama_model = LLM(
    model="ollama/llama3.2",
    base_url="http://localhost:11434"  # URL for Ollama's local server
)



---

## 4. **Agent Initialization**
- **`web_crawler_agent`**: This agent performs the task of gathering information from the web using a query and the Serper API for search results.
- **`summarizer_agent`**: This agent summarizes the web data gathered by the `web_crawler_agent`.
- **`report_generator_agent`**: This agent generates a final research report from the summarized data, tailored to the selected report style.


In [2]:
# Initialize the agents
web_crawler_agent = WebCrawlerAgent(
    llm=llama_model,
    role="web crawler",
    backstory="I search the web to gather information on a given topic.",
    goal="Fetch web search results for a given query.",
    serper_api_key=SERPER_API_KEY
)


summarizer_agent = SummarizerAgent(llama_model)

report_generator_agent = ReportGeneratorAgent(llm=llama_model)




---

## 5. **Define Report Generation Logic**
The `generate_report()` function orchestrates the entire process of generating a research report:
1. Defines three tasks for the workflow:
   - **Web Crawler Task**: Fetches data from the web.
   - **Summarizer Task**: Summarizes the fetched data.
   - **Report Task**: Generates a comprehensive research report.
2. A `Crew` is created to manage the agents and tasks, specifying the order in which tasks are executed.
3. The tasks are executed sequentially, and the result is returned as a formatted research report in the chosen style.


---

## 6. **Gradio User Interface Setup**
- **UI Elements**:
  - **Markdown**: Displays headings and instructions.
  - **Textbox (`query_input`)**: Allows the user to input a query.
  - **Radio (`report_style`)**: Lets the user select the desired style of the report (e.g., Detailed Analysis, Brief Summary, Bullet Points).
  - **Button (`generate_button`)**: When clicked, it triggers the report generation process.
  - **Markdown (`report_output`)**: Displays the final generated report.

- **Button Click Event**: The button triggers the `generate_report()` function, passing user inputs and showing the output.

- **Launch Gradio App**: The app is launched using `demo.launch(share=True)` to make it accessible to users.

---

## 7. **Final Workflow Explanation**
1. The user inputs a query and selects a report style.
2. The user clicks the "Generate Report" button, initiating the task sequence.
3. The tasks are executed in order:
   - Web scraping with the `web_crawler_agent`.
   - Summarization with the `summarizer_agent`.
   - Report generation with the `report_generator_agent`.
4. The result is displayed as a formatted report in the Gradio interface.


In [4]:


# Modified generate_report function to include report style
def generate_report(query, report_style):
    # Task definitions
    web_crawler_task = Task(
        description=query, 
        expected_output="A detailed response about the query.",
        agent=web_crawler_agent
    )

    summarizer_task = Task(
        description="Summarize the web data.", 
        expected_output="Concise summaries of the search results.", 
        agent=summarizer_agent
    )

    report_task = Task(
        description=f"Generate a final research report in '{report_style}' style by removing bias if any.",
        expected_output="A comprehensive research report.",
        agent=report_generator_agent
    )

    # Create the Crew
    crew = Crew(
        agents=[web_crawler_agent, summarizer_agent, report_generator_agent],
        tasks=[web_crawler_task, summarizer_task, report_task],
        verbose=True,
        process=Process.sequential,
        context_flow={
            "summarizer_task": "web_crawler_task",  # Output of web_crawler_task goes to summarizer_task
            "report_task": "summarizer_task",      # Output of summarizer_task goes to report_task
        }
    )

    # Execute tasks and return the final output
    result = crew.kickoff()
    formatted_report = f"###### Report Generated in {report_style} Style:\n\n{result.raw}"

    # Return the formatted report
    return formatted_report

# Gradio Interface
with gr.Blocks() as demo:
    # Improved visuals using markdown and layout changes
    gr.Markdown("## 🚀 Query-Based Research Report Generator")
    gr.Markdown("#### Enter your query and choose the report style to generate a detailed, formatted report!")

    with gr.Row():
        query_input = gr.Textbox(
            label="Enter Your Query", 
            placeholder="e.g., Recent advancements in Indian Agriculture",
            lines=2,
            elem_id="query-input"
        )

    with gr.Row():
        report_style = gr.Radio(
            choices=[ "Detailed Analysis", "Brief Summary","Bullet Points"],
            label="Select Report Style",
            value="Detailed Analysis"
        )
    
    with gr.Row():
        generate_button = gr.Button("Generate Report", elem_id="generate-button", variant="primary")
    
    with gr.Row():
        report_output = gr.Markdown(
            label="Generated Report"
        )
    
    generate_button.click(
        generate_report, 
        inputs=[query_input, report_style], 
        outputs=[report_output]
    )

# Launch the Gradio app
demo.launch(share=True)